<a href="https://colab.research.google.com/github/victoria2012/test_deeplearning/blob/master/NaverSentimentAnalysis_loadmodel_LSTM_t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
loaded_model = tf.keras.models.load_model('./NaverSentimentAnalysis_LSTM_t.h5')
loaded_model

##### 과정 : 분류기 --> 불용어 제거(stopwords) --> 사전을 근거로 숫자화 --> padding --> predict

In [4]:
!python -m pip install konlpy

     |████████████████████████████████| 19.4 MB 5.3 MB/s 
     |████████████████████████████████| 448 kB 59.4 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
import konlpy

In [6]:
okt = konlpy.tag.Okt()    # 태그 분류기 초기화

In [9]:
import pickle

In [10]:
stopwords = pickle.load(open('./stopwords.pkl','rb'))
print(stopwords)

['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']


In [ ]:
# 불용어 처리의 다른 방법을 사용해 보자 
# words = list()
# for tok in temp_x:
#   if tok not in stopwords:
#     words.append(tok)

In [17]:
# 감독이 예술이네. --> 웹 사용자 입력한 문자는 컴퓨터가 바로 해석할 수 없기 때문에 숫자로 바꿔서 넣어줘야 함. 긍정인지 부정인지 확인. 
new_sentence = okt.morphs('감독이 예술이네.', stem=True)   # 단어를 분류하고 원형을 찾아주는 작업. 딥러닝의 상태와 똑같이 만들어주기 위해 stem=True 넣어줌
new_sentence = [tok for tok in new_sentence if tok not in stopwords]   # 리스트를 의미하는 대괄호[] 안에 for문과 if문을 넣어주고 맨 왼쪽에는 저장할 값을 지정
new_sentence

['감독', '예술', '이네', '.']

In [18]:
# pickle을 불러와서 토큰화하는 작업 진행
tokenizer = pickle.load(open('./tokenizer.pkl','rb'))     # rb는 read binary의 뜻
tokenizer

In [19]:
vob_size = len(tokenizer.word_index)
vob_size

1572

In [20]:
encoded = tokenizer.texts_to_sequences([new_sentence])    # 대괄호 안에 넣어줘야 함.
encoded

[[92, 128, 1]]

In [23]:
pad_new = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=50)
pad_new

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,  92, 128,   1]],
      dtype=int32)

In [25]:
# ['감독이 예술이네.']
score = loaded_model.predict(pad_new)    # 무엇이 무엇을 predict하는가? 영어의 문법과 연관해서 생각해 볼 것 
score

array([[0.49271348]], dtype=float32)

In [27]:
if(score > 0.5):
  print('긍정', score*100)
else:
  print('부정', score*100)

부정 [[49.271347]]


In [ ]:
okt = konlpy.tag.Okt()
new_sentence = okt.morphs('감독이 예술이네.', stem=True)
new_sentence = [ tok for tok in new_sentence if tok not in stopwords ]
encoded = tokenizer.texts_to_sequences([new_sentence])
pad_new = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=50)
score = loaded_model.predict(pad_new)

In [28]:
def sentiment_predict(sentence):
  okt = konlpy.tag.Okt()
  new_sentence = okt.morphs(sentence, stem=True)
  new_sentence = [ tok for tok in new_sentence if tok not in stopwords ]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=50)
  score = loaded_model.predict(pad_new)
  return score

In [32]:
words = '이 영화 꽝이네.'
sentiment_predict(words)

array([[0.49839693]], dtype=float32)

In [33]:
words = '액션도 좋고 줄거리도 재밌어요.'
sentiment_predict(words)

array([[0.4999679]], dtype=float32)